In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re

from path_fns.filepaths import (
    TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON,
    DEDUPE_OUTPUTS_FILES_BASE,
    FINAL_CORRUPTED_OUTPUT_FILES_BASE,
)

import pandas as pd
import numpy as np
import os

from graphs.linkagegraph import LinkageGraph
import igraph as ig

In [2]:
master_data = pd.read_parquet(TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON)
len(master_data)

311915

In [3]:
master_data.head(5)


,human,dod,family_name,dob,given_name,country_citizen,occupation,occupationLabel,country_citizenLabel,sex_or_genderLabel,...,ethnicityLabel,birth_coordinates,residence_coordinates,humanLabel,humanAltLabel,birth_name,given_nameLabel,family_nameLabel,full_name_arr,group_id
0,0,[1953-01-01],[Q1544833],[1876-01-01],[Q4925477],[],"[Q639669, Q158852, Q36834]","[musician, conductor, composer]",[],[male],...,[],[],[],[john a. greenwood],[],[],[john],[greenwood],"[john greenwood, john a. greenwood]",3
1,1,[1953-01-01],[Q1965022],[1891-01-01],"[Q18145837, Q632842]",[],"[Q170790, Q169470]","[mathematician, physicist]",[],[male],...,[],[],[],[hans adolf bauer],[],[],"[adolf, hans]",[bauer],"[adolf hans bauer, hans adolf bauer]",0
2,2,[1953-01-01],[Q187733],[1879-01-01],[Q4925932],[],"[Q333634, Q37226]","[translator, teacher]",[],[male],...,[],[],[],[franz zeller],[],[],[franz],[zeller],[franz zeller],3
3,3,[1953-01-01],[Q82014711],[1880-01-01],[Q2648973],[],[Q3330547],[chronicler],[],[male],...,[],[],[],[q112403983],[],[],[františek],[hurdálek],[františek hurdálek],2
4,4,[1953-01-01],[Q21452861],[1875-01-01],[Q12173670],[],[],[],[],[male],...,[],[],[],[q112367924],[],[],[jan],[menčík],[jan menčík],4


In [17]:
# master_data.groupby("sex_or_genderLabel").count()
df_size = master_data[["given_nameLabel", "family_nameLabel", "sex_or_genderLabel", "dob", "country_citizen"]].applymap(len)

df_size.groupby("country_citizen").count()

,given_nameLabel,family_nameLabel,sex_or_genderLabel,dob
country_citizen,,,,
0,55032,55032,55032,55032
1,212690,212690,212690,212690
2,39227,39227,39227,39227
3,3837,3837,3837,3837
4,881,881,881,881
5,161,161,161,161
6,59,59,59,59
7,21,21,21,21
8,6,6,6,6


In [ ]:
corrupted_data_filename = "max_corruptions-99_prob_mult-0.5_set-0.parquet"

corrupted_data_path = os.path.join(
        FINAL_CORRUPTED_OUTPUT_FILES_BASE, 
        corrupted_data_filename
    )

corrupted_records = pd.read_parquet(corrupted_data_path)
len(corrupted_records)

In [ ]:
corrupted_records.head(5)

In [4]:
dedupe_data_filename = "max_corruptions-49_prob_mult-2.0_set-0.parquet"

dedupe_data_path = os.path.join(
        DEDUPE_OUTPUTS_FILES_BASE, 
        dedupe_data_filename
    )

dedupe_records = pd.read_parquet(dedupe_data_path)
len(dedupe_records)


45690008

In [4]:
dedupe_records.head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,bf_given_name,bf_family_name,bf_dob_d,bf_dob_m,bf_dob_y,bf_gender,cluster_l,cluster_r
0,8.227130,0.996674,76024016,76024018,385.525695,4462.898819,0.009815,9.574293,62.027934,1.288185,76024,76024
1,19.056766,0.999998,76025000,76025018,385.525695,4462.898819,17.862896,9.574293,62.027934,1.288185,76025,76025
2,19.056766,0.999998,76025001,76025018,385.525695,4462.898819,17.862896,9.574293,62.027934,1.288185,76025,76025
3,19.056766,0.999998,76025002,76025018,385.525695,4462.898819,17.862896,9.574293,62.027934,1.288185,76025,76025
4,19.056766,0.999998,76025003,76025018,385.525695,4462.898819,17.862896,9.574293,62.027934,1.288185,76025,76025


In [5]:
def get_weight_corresponding_max_fmeasure(df, sample = None):
    if sample is not None:
        df = df.sample(n=sample, axis=0, ignore_index=True).copy()
    else:
        df = df.copy()
    
    match_weights = sorted(df.match_probability.unique())

    f_measures = []
    for match_weight in match_weights:
        true_positives = float(len(df.loc[(df["match_weight"] >= match_weight) & (df["cluster_l"] == df["cluster_r"])]))
        false_positives = float(len(df.loc[(df["match_weight"] >= match_weight) & (df["cluster_l"] != df["cluster_r"])]))
        false_negatives = float(len(df.loc[(df["match_weight"] < match_weight) & (df["cluster_l"] == df["cluster_r"])]))

        f_measures.append(true_positives / (true_positives + 0.5 * (false_positives + false_negatives)))

    max_fmeasure_index = f_measures.index(max(f_measures))

    return match_weights[max_fmeasure_index]

In [7]:
threshold = get_weight_corresponding_max_fmeasure(dedupe_records, int(len(dedupe_records)/50))
print(threshold)

0.7647495712808775


In [16]:
# threshold = 0.9897444028365115
len(dedupe_records.loc[(dedupe_records["match_weight"] >= threshold) & (dedupe_records["cluster_l"] != dedupe_records["cluster_r"])]) / len(dedupe_records)

0.025683843172012576

In [14]:
graphs_all = LinkageGraph(dedupe_records[0:1000000], prob_threshold = threshold, min_order = 1)

In [22]:
df_gmeasures = graphs_all.get_measures()

In [10]:
len(graphs_all.connected_graphs)

594

In [11]:
len(df_gmeasures)

594

In [16]:
len(df_gmeasures.loc[(df_gmeasures['vertices'] <= 50) & (df_gmeasures['any_false_links'] == True)])

8

In [18]:
len(df_gmeasures.loc[(df_gmeasures['vertices'] <= 50) & (df_gmeasures['any_false_links'] == True)]) / len(df_gmeasures.loc[(df_gmeasures['vertices'] <= 50)])

0.0017997750281214847

In [23]:
df_gmeasures.loc[(df_gmeasures['vertices'] <= 50) & (df_gmeasures['any_false_links'] == True)].head(25)

,vertices,edges,diameter,diameter_weight_total,diameter_weight_given_name,diameter_weight_family_name,diameter_weight_dob_d,diameter_weight_dob_m,diameter_weight_dob_y,diameter_weight_gender,transitivity,tri_cluster_coef,assortativity_degree,density,false_link_count,distinct_entities,any_false_links
67,33,358,2,0.063164,0.208907,0.000648,0.106228,0.189338,1.957943,1.355729,0.726468,0.854594,-0.587151,0.678030,238,2,True
810,25,69,2,0.010684,0.005374,0.000648,0.106228,0.189338,1.957943,0.874255,0.224832,0.899565,-0.916667,0.230000,66,2,True
901,34,408,2,0.010684,0.031696,0.000648,0.106228,0.189338,1.957943,0.874255,0.769231,0.863636,-0.545455,0.727273,288,2,True
1601,12,35,2,0.000538,0.386119,0.000648,0.106228,0.189338,0.031932,0.874255,0.000000,0.000000,-1.000000,0.530303,35,2,True
2456,24,66,2,0.063164,0.031696,0.000648,0.106228,0.189338,1.957943,0.874255,0.233577,0.896245,-0.913043,0.239130,63,2,True
2457,2,1,1,0.005342,0.002687,0.000324,0.053114,0.094669,0.978972,0.437128,NaN,NaN,NaN,1.000000,1,2,True
2464,37,642,2,0.031851,0.208907,0.000648,0.106228,0.189338,0.994938,0.874255,0.986853,0.987645,-0.064924,0.963964,288,2,True
3505,37,623,2,0.063164,0.031696,0.000648,0.106228,1.085423,1.957943,1.355729,0.945638,0.951460,-0.242717,0.935435,217,2,True


In [76]:
#graphs_all.plot_subgraph(72, "/mnt/sdd/dev/cluster_example.pdf")

'Image saved.'